# Setting 

In [15]:
import os
import glob
import random
import time
import math
import logging 
from tqdm import tqdm

import torch

import torch.nn as nn
import torch.optim as optim
import pickle
import copy, math
from torch.utils.data import DataLoader, Subset

from transformers import BertTokenizer
from sklearn.preprocessing import LabelEncoder

import torch.nn.functional as F
from torch.nn.modules.utils import _pair
from torch.nn.parameter import Parameter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image

import torchvision.transforms as T

import models.shared_perceiver as sp
import models.layers as nl

In [2]:
device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def seed_everything(seed):
    torch.manual_seed(seed) #torch를 거치는 모든 난수들의 생성순서를 고정한다
    torch.cuda.manual_seed(seed) #cuda를 사용하는 메소드들의 난수시드는 따로 고정해줘야한다 
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True #딥러닝에 특화된 CuDNN의 난수시드도 고정 
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed) #numpy를 사용할 경우 고정
    random.seed(seed) #파이썬 자체 모듈 random 모듈의 시드 고정

def seed_worker(worker_id): #데이터로더 난수고정
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

seed_everything(42)
g = torch.Generator()
g.manual_seed(42)
NUM_WORKERS = 4 # 서브프로세스관리자 수. 난수생성과 관련있습니다. 일단은 4로 고정합니다.


# Load Models

## Params & Hyperparams

In [4]:
FINETUNE_EPOCHS = 50
NETWORK_WIDTH_MULTIPLIER = 1.0
MAX_NETWORK_WIDTH_MULTIPLIER = 2.0

LR = 1e-2
LR_MASK = 1e-4
WEIGHT_DECAY = 4e-5
BATCH_SIZE = 32
TOTAL_NUM_TASKS = 6

NUM_CLASSES = 3

EMBED_DIM = 128
LATENT_DIM = 64
LATENT_SIZE = 64
NUM_BLOCKS = 4


task_id = 1
target_id = 7

In [5]:
data_path = '/home/youlee/n24news/n24news'

file_path = '/home/Minju/Perceiver'

groups_path = data_path + '/captions_and_labels.csv'

model_path = file_path + '/shared_layer_model/'
loader_path = file_path + '/shared_layer_loader/'

save_folder = file_path + '/finetune/'
load_folder = file_path + f'/{task_id}/{target_id}'

In [ ]:
# version_name = 'CPG_fromsingle_scratch_woexp_target'
# single_version_name = 'CPG_single_scratch_woexp'
# baseline_file = 'logs_lenet5/baseline_cifar100_acc_scratch.txt'
# checkpoints_name = 'checkpoints_lenet5'

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
MAX_LENGTH=128

## 모델 불러오기

In [11]:
model_dir = os.path.join(model_path, f"text_model_{task_id}.pth.tar")
if not os.path.exists(model_dir):
    raise FileNotFoundError(f"모델 체크포인트를 찾을 수 없습니다: {model_dir}")

checkpoint = torch.load(model_dir, map_location=device)
model = sp.Perceiver(input_dim=EMBED_DIM, latent_dim=LATENT_DIM, 
                     latent_size=LATENT_SIZE, num_classes=NUM_CLASSES,
                   num_blocks=NUM_BLOCKS, self_attn_layers_per_block=1)

model = sp.CombinedModel(vocab_size=tokenizer.vocab_size, embed_dim=EMBED_DIM, perceiver_model=model)

model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)

  


/tmp/ipykernel_1729873/2377390368.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_dir, map_location=device)


## 데이터로더

In [13]:
file_path = '/home/youlee/n24news/n24news/captions_and_labels.csv'
data = pd.read_csv(file_path)

groups = [
    ["Opinion", "Art & Design", "Television"],
    ["Music", "Travel", "Real Estate"],
    ["Books", "Theater", "Health"],
    ["Sports", "Science", "Food"],
    ["Fashion & Style", "Movies", "Technology"],
    ["Dance", "Media", "Style"]
]

output_paths = []
for i, group_labels in enumerate(groups, 1):
    group_data = data[data['Label'].isin(group_labels)]
    output_path = f'/home/youlee/n24news/n24news/regroup_{i}.csv'
    group_data.to_csv(output_path, index=False)
    output_paths.append(output_path)

print("생성된 그룹별 CSV 파일 경로:")
for path in output_paths:
    print(path)

생성된 그룹별 CSV 파일 경로:
/home/youlee/n24news/n24news/regroup_1.csv
/home/youlee/n24news/n24news/regroup_2.csv
/home/youlee/n24news/n24news/regroup_3.csv
/home/youlee/n24news/n24news/regroup_4.csv
/home/youlee/n24news/n24news/regroup_5.csv
/home/youlee/n24news/n24news/regroup_6.csv


In [19]:
for idx, group_file in enumerate(output_paths, start=1):
    if idx != task_id:
        continue
    print(f"\ngroup {idx} 처리 중...")

    df = pd.read_csv(group_file)
    label_encoder = LabelEncoder()
    df['Label'] = label_encoder.fit_transform(df['Label'])
    num_classes = len(label_encoder.classes_)

    input_ids, attention_masks = sp.tokenize_data(df, tokenizer=tokenizer, MAX_LENGTH=MAX_LENGTH)
    labels = torch.tensor(df['Label'].values)

    dataset = sp.CustomDataset(input_ids, attention_masks, labels)
    sample = dataset [0]
    print(sample)
    print(len(dataset)) 
 


group 1 처리 중...
{'input_ids': tensor([ 101, 1996, 2317, 2160, 2003, 1037, 4556, 2806, 3538, 1997, 4294, 1010,
        4427, 2011, 3306, 1998, 3142, 3121, 1012,  102,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0,

In [20]:
indices = torch.randperm(len(dataset))  # 랜덤하게 인덱스를 섞음
train_size = int(0.8 * len(dataset))
train_indices = indices[:train_size]
test_indices = indices[train_size:]

train_subset = Subset(dataset, train_indices.tolist())
test_subset = Subset(dataset, test_indices.tolist())

train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True,
                        num_workers=NUM_WORKERS, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_subset, batch_size=BATCH_SIZE, shuffle=False,
                        num_workers=NUM_WORKERS, worker_init_fn=seed_worker, generator=g)


In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY, momentum=0.9)

In [24]:
# -------------------- #
# 🔹 Fine-tuning 수행
# -------------------- #
for epoch in range(FINETUNE_EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{FINETUNE_EPOCHS}"):
        # inputs, labels = inputs.to(device), labels.to(device)
        inputs = batch['input_ids'].to(device)
        attention_masks = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)            
        correct += predicted.eq(labels).sum().item()

    train_acc = 100. * correct / total
    logging.info(f"Epoch [{epoch+1}/{FINETUNE_EPOCHS}], Loss: {running_loss:.4f}, Train Acc: {train_acc:.2f}%")

        # -------------------- #
        # 🔹 모델 저장
        # -------------------- #
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    torch.save({'state_dict': model.state_dict()}, os.path.join(save_folder, "fine_tuned_model.pth.tar"))

logging.info("Fine-tuning 완료!")

Epoch 50/50: 100%|██████████| 183/183 [00:02<00:00, 66.30it/s]
